In [3]:
import pandas as pd


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Đọc file Excel và load sheet "Dulieu0205"
df=pd.read_excel('Processed_THONGKESVISHOCTUCHON.xlsx',sheet_name='Dulieu0205')
# Lấy danh sách sinh viên duy nhất
unique_students = df['IDSinhVien'].unique()

# Chọn một sinh viên để dự đoán (ví dụ: sinh viên đầu tiên)
selected_student_id = unique_students[0]
student_data = df[df['IDSinhVien'] == selected_student_id]

# Chỉ giữ lại các cột cần thiết cho việc dự đoán: 'MaMonHoc', 'HK', 'DiemTongKet'
student_data = student_data[['MaMonHoc', 'HK', 'DiemTongKet']]

# Mã hóa cột 'MaMonHoc' thành các giá trị số
label_encoder = LabelEncoder()
student_data['MaMonHoc_Encoded'] = label_encoder.fit_transform(student_data['MaMonHoc'])

# Chuẩn bị dữ liệu đầu vào (features) và nhãn (labels)
X = student_data[['HK', 'MaMonHoc_Encoded']]
y = student_data['DiemTongKet']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% huấn luyện, 20% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo các mô hình hồi quy
regression_models = {
    "Random Forest Regressor": RandomForestRegressor(),
    "SVR": SVR(),
    "Hồi quy tuyến tính": LinearRegression()
}

# Huấn luyện các mô hình và tính toán Mean Squared Error (MSE)
mse_scores = {}

for model_name, model in regression_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores[model_name] = mse

# Dự đoán các môn học cho học kỳ tiếp theo
next_semester_hk = student_data['HK'].max() + 1
predicted_scores = regression_models["Random Forest Regressor"].predict(
    [[next_semester_hk, i] for i in range(len(label_encoder.classes_))]
)

# Lấy 2 môn học có điểm dự đoán cao nhất
top_2_predicted = sorted(zip(predicted_scores, label_encoder.inverse_transform(range(len(predicted_scores)))), reverse=True)[:2]

# In ra kết quả
mse_scores, top_2_predicted

c:\Users\ICT\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


({'Random Forest Regressor': 0.36463399999999635,
  'SVR': 0.27350445814152724,
  'Hồi quy tuyến tính': 0.4461635818382685},
 [(8.531999999999998, 3893), (8.430000000000001, 901)])